In [1]:
import cv2
import os

# Paths
video_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/hurdling/stages/stage3/videos"
frames_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/hurdling/stages/stage3/frames"

# Ensure output directory exists
os.makedirs(frames_dir, exist_ok=True)

def extract_frames(video_path, output_dir, interval=1):
    """
    Extract frames from a video at a given interval (default: every 1 frame).
    """
    cap = cv2.VideoCapture(video_path)
    count = 0
    frame_count = 0
    video_name = os.path.basename(video_path).rsplit('.', 1)[0]  # Get the video name without extension

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Save every 'interval' frame
        if count % interval == 0:
            frame_path = os.path.join(output_dir, f"{video_name}_frame{frame_count}.jpg")
            cv2.imwrite(frame_path, frame)
            frame_count += 1

        count += 1

    cap.release()

# Process all videos
for video_file in os.listdir(video_dir):
    # Check if the filename ends with ".mp4" and starts with a valid number
    if video_file.endswith(".mp4"):
        try:
            # Split by '_' and attempt to convert the first part to float
            prefix = video_file.split('_')[0]
            float(prefix)  # This ensures it works for values like '0.5', '1', or '0'
            
            # Proceed with frame extraction
            video_path = os.path.join(video_dir, video_file)
            video_frames_dir = os.path.join(frames_dir, video_file.rsplit('.', 1)[0])  # Use full name
            os.makedirs(video_frames_dir, exist_ok=True)
            extract_frames(video_path, video_frames_dir)
            print(f"Frames extracted for {video_file}")
        except ValueError:
            print(f"Skipping file with invalid prefix: {video_file}")

print("Frame extraction completed!")

Frames extracted for 1_user2.mp4
Frames extracted for 1_user9_2.mp4
Frames extracted for 1_user5.mp4
Frames extracted for 1_user13.mp4
Frames extracted for 1_user21_2.mp4
Frames extracted for 0_user3.mp4
Frames extracted for 0_user1.mp4
Frames extracted for 0_user3_2.mp4
Frames extracted for 0_user10.mp4
Frames extracted for 1_user2_2.mp4
Frames extracted for 0_user10_2.mp4
Frames extracted for 1_user21.mp4
Frames extracted for 1_user9.mp4
Frames extracted for 1_user5_2.mp4
Frames extracted for 1_user19.mp4
Frames extracted for 1_user13_2.mp4
Skipping file with invalid prefix: 0116.mp4
Frame extraction completed!


In [2]:
import cv2
import os
import mediapipe as mp
import json
import math

# Paths
frames_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/hurdling/stages/stage3/frames"
keypoints_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/hurdling/stages/stage3/keypoints"

# Ensure output directory exists
os.makedirs(keypoints_dir, exist_ok=True)

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, model_complexity=2, enable_segmentation=False)

# Thresholds for detecting fully extended lead leg and hurdle clearance
HURDLE_CLEARANCE_THRESHOLD_Y = 0.3  # Y-coordinate threshold for foot height (can be adjusted based on hurdle height)
LEG_EXTENSION_ANGLE_THRESHOLD = 170  # Angle threshold for full extension (degrees)

def extract_keypoints_from_frame(frame_path):
    """
    Extract keypoints relevant to posture and alignment to detect when the lead leg is fully extended.
    """
    image = cv2.imread(frame_path)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = pose.process(rgb_image)

    if result.pose_landmarks:
        # Extract relevant keypoints for detecting lead leg extension and hurdle clearance
        keypoints = {
            "right_foot_index": {
                "x": result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].x,
                "y": result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].y,
                "z": result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].z,
            },
            "left_foot_index": {
                "x": result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].x,
                "y": result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].y,
                "z": result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].z,
            },
            "right_knee": {
                "x": result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE].x,
                "y": result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE].y,
                "z": result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE].z,
            },
            "left_knee": {
                "x": result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].x,
                "y": result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].y,
                "z": result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].z,
            },
            "right_hip": {
                "x": result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP].x,
                "y": result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP].y,
                "z": result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP].z,
            },
            "left_hip": {
                "x": result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].x,
                "y": result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].y,
                "z": result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].z,
            },
        }
        
        return keypoints
    return None

def calculate_angle(a, b, c):
    """
    Calculate the angle between three points (a, b, c) where b is the vertex.
    Angle is in degrees.
    """
    ba = [a[0] - b[0], a[1] - b[1]]
    bc = [c[0] - b[0], c[1] - b[1]]
    
    dot_product = ba[0] * bc[0] + ba[1] * bc[1]
    magnitude_a = math.sqrt(ba[0] ** 2 + ba[1] ** 2)
    magnitude_b = math.sqrt(bc[0] ** 2 + bc[1] ** 2)
    
    cos_angle = dot_product / (magnitude_a * magnitude_b)
    angle = math.acos(cos_angle) * (180.0 / math.pi)
    
    return angle

def detect_leg_extension_and_hurdle_clearance(keypoints_sequence):
    """
    Detect if the lead leg is fully extended and the hurdle has been cleared.
    """
    fully_extended_and_clear = False

    for keypoints in keypoints_sequence:
        # Extract keypoints for lead leg (right leg)
        right_foot = keypoints["right_foot_index"]
        right_knee = keypoints["right_knee"]
        right_hip = keypoints["right_hip"]
        
        # Calculate knee angle (using right hip, right knee, and right ankle)
        hip_knee_angle = calculate_angle(
            [right_hip["x"], right_hip["y"]],
            [right_knee["x"], right_knee["y"]],
            [right_foot["x"], right_foot["y"]]
        )

        # Check if knee angle is near 180 degrees (fully extended) and if the foot clears the hurdle
        if hip_knee_angle > LEG_EXTENSION_ANGLE_THRESHOLD and right_foot["y"] < HURDLE_CLEARANCE_THRESHOLD_Y:
            fully_extended_and_clear = True
            break

    return fully_extended_and_clear

def process_frames(video_frames_dir, output_path):
    """
    Process all frames for a video, detect leg extension and hurdle clearance, and save keypoints as JSON.
    """
    keypoints_data = []
    for frame_file in sorted(os.listdir(video_frames_dir)):
        frame_path = os.path.join(video_frames_dir, frame_file)
        keypoints = extract_keypoints_from_frame(frame_path)
        if keypoints:
            keypoints_data.append(keypoints)

    # Detect if the lead leg is fully extended and the hurdle is cleared
    leg_extended_and_clear = detect_leg_extension_and_hurdle_clearance(keypoints_data)

    # Save keypoints and leg extension status to JSON
    output_data = {
        "keypoints": keypoints_data,
        "lead_leg_fully_extended_and_hurdle_cleared": leg_extended_and_clear,
    }
    with open(output_path, 'w') as f:
        json.dump(output_data, f, indent=4)

# Process frames for each video
for video_name in os.listdir(frames_dir):
    video_frames_dir = os.path.join(frames_dir, video_name)
    if os.path.isdir(video_frames_dir):
        output_path = os.path.join(keypoints_dir, f"{video_name}_keypoints.json")
        process_frames(video_frames_dir, output_path)
        print(f"Keypoints and leg extension with hurdle clearance detected for {video_name}")

print("Processing completed!")

I0000 00:00:1737033945.595569 4930217 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1737033945.650847 4930877 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737033945.677649 4930877 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737033945.727454 4930878 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Keypoints and leg extension with hurdle clearance detected for 0_user3
Keypoints and leg extension with hurdle clearance detected for 1_user13
Keypoints and leg extension with hurdle clearance detected for 1_user5
Keypoints and leg extension with hurdle clearance detected for 1_user2
Keypoints and leg extension with hurdle clearance detected for 0_user3_2
Keypoints and leg extension with hurdle clearance detected for 0_user10
Keypoints and leg extension with hurdle clearance detected for 1_user9_2
Keypoints and leg extension with hurdle clearance detected for 0_user10_2
Keypoints and leg extension with hurdle clearance detected for 0_user1
Keypoints and leg extension with hurdle clearance detected for 1_user21_2
Keypoints and leg extension with hurdle clearance detected for 1_user2_2
Keypoints and leg extension with hurdle clearance detected for 1_user21
Keypoints and leg extension with hurdle clearance detected for 1_user19
Keypoints and leg extension with hurdle clearance detected fo

In [3]:
import os
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image

# Paths
frames_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/hurdling/stages/stage3/frames"
clip_features_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/hurdling/stages/stage3/clip_features"
os.makedirs(clip_features_dir, exist_ok=True)

# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def extract_clip_features(video_frames_dir, output_path, batch_size=8):
    """
    Extract CLIP features for all frames in a video using batch processing.
    """
    frame_features = []
    frame_paths = sorted(os.listdir(video_frames_dir))
    images = []

    for i, frame_file in enumerate(frame_paths):
        try:
            frame_path = os.path.join(video_frames_dir, frame_file)
            image = Image.open(frame_path).convert("RGB")
            images.append(image)

            # Process batch
            if len(images) == batch_size or i == len(frame_paths) - 1:
                inputs = processor(images=images, return_tensors="pt", padding=True).to(device)
                with torch.no_grad():
                    image_features = model.get_image_features(**inputs).cpu().numpy()
                    frame_features.extend(image_features)
                images = []  # Clear batch to free memory
        except Exception as e:
            print(f"Error processing frame {frame_file}: {e}")
            continue

    # Save features
    torch.save(frame_features, output_path)

# Process all videos
for video_name in os.listdir(frames_dir):
    video_frames_dir = os.path.join(frames_dir, video_name)
    if os.path.isdir(video_frames_dir):
        output_path = os.path.join(clip_features_dir, f"{video_name}_clip.pt")
        extract_clip_features(video_frames_dir, output_path)
        print(f"CLIP features extracted for {video_name}")

print("CLIP feature extraction completed!")

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CLIP features extracted for 0_user3
CLIP features extracted for 1_user13
CLIP features extracted for 1_user5
CLIP features extracted for 1_user2
CLIP features extracted for 0_user3_2
CLIP features extracted for 0_user10
CLIP features extracted for 1_user9_2
CLIP features extracted for 0_user10_2
CLIP features extracted for 0_user1
CLIP features extracted for 1_user21_2
CLIP features extracted for 1_user2_2
CLIP features extracted for 1_user21
CLIP features extracted for 1_user19
CLIP features extracted for 1_user13_2
CLIP features extracted for 1_user9
CLIP features extracted for 1_user5_2
CLIP feature extraction completed!


In [4]:
import json
import os
import torch
import numpy as np
import math

# Paths
keypoints_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/hurdling/stages/stage3/keypoints"
clip_features_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/hurdling/stages/stage3/clip_features"
combined_features_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/hurdling/stages/stage3/combined_features"

# Ensure output directory exists
os.makedirs(combined_features_dir, exist_ok=True)

# Threshold for detecting hurdle clearance
HURDLE_CLEARANCE_THRESHOLD_Y = 0.3  # Example threshold for foot height (this can be adjusted based on hurdle height)
LEG_EXTENSION_ANGLE_THRESHOLD = 170  # Angle threshold for knee extension (degrees)

def calculate_angle(a, b, c):
    """
    Calculate the angle between three points (a, b, c) where b is the vertex.
    Angle is in degrees.
    """
    ba = [a[0] - b[0], a[1] - b[1]]
    bc = [c[0] - b[0], c[1] - b[1]]
    
    dot_product = ba[0] * bc[0] + ba[1] * bc[1]
    magnitude_a = math.sqrt(ba[0] ** 2 + ba[1] ** 2)
    magnitude_b = math.sqrt(bc[0] ** 2 + bc[1] ** 2)
    
    cos_angle = dot_product / (magnitude_a * magnitude_b)
    angle = math.acos(cos_angle) * (180.0 / math.pi)
    
    return angle

def detect_leg_extension_and_hurdle_cleared(keypoints_sequence):
    """
    Detect if the lead leg is fully extended and the hurdle has been cleared.
    """
    lead_leg_extended_and_clear = False

    for keypoints in keypoints_sequence:
        right_foot = keypoints["right_foot_index"]
        right_knee = keypoints["right_knee"]
        right_hip = keypoints["right_hip"]
        
        # Calculate knee angle (using right hip, right knee, and right ankle)
        hip_knee_angle = calculate_angle(
            [right_hip["x"], right_hip["y"]],
            [right_knee["x"], right_knee["y"]],
            [right_foot["x"], right_foot["y"]]
        )

        # Check if knee angle is near 180 degrees (fully extended) and if the foot clears the hurdle
        if hip_knee_angle > LEG_EXTENSION_ANGLE_THRESHOLD and right_foot["y"] < HURDLE_CLEARANCE_THRESHOLD_Y:
            lead_leg_extended_and_clear = True
            break

    return lead_leg_extended_and_clear

def combine_features(video_name, pose_path, clip_path, output_path):
    """
    Combine pose-based features and CLIP embeddings for a video.
    """
    # Load pose-based features
    with open(pose_path, 'r') as f:
        keypoints_data = json.load(f)["keypoints"]

    # Detect lead leg extension and hurdle clearance
    lead_leg_extended_and_clear = detect_leg_extension_and_hurdle_cleared(keypoints_data)

    # Load CLIP embeddings
    clip_data = torch.load(clip_path)

    # Combine features for each frame
    combined_data = []
    for i, clip_frame in enumerate(clip_data):
        # Pose features indicating whether the lead leg is fully extended and the hurdle is cleared
        pose_features = {
            "lead_leg_extended_and_cleared": 1 if lead_leg_extended_and_clear else 0,  # Binary flag for leg extension and hurdle clearance
        }
        combined_frame = np.concatenate([clip_frame, list(pose_features.values())])
        combined_data.append(combined_frame)

    # Save combined features
    torch.save(combined_data, output_path)

# Process all videos
for video_name in os.listdir(keypoints_dir):
    if video_name.endswith("_keypoints.json"):
        video_name_base = video_name.replace("_keypoints.json", "")
        pose_path = os.path.join(keypoints_dir, video_name)
        clip_path = os.path.join(clip_features_dir, f"{video_name_base}_clip.pt")
        output_path = os.path.join(combined_features_dir, f"{video_name_base}_combined.pt")

        if os.path.exists(clip_path):
            combine_features(video_name_base, pose_path, clip_path, output_path)
            print(f"Combined features saved for {video_name_base}")

print("Feature combination completed!")

Combined features saved for 0_user3
Combined features saved for 1_user13_2
Combined features saved for 1_user9
Combined features saved for 1_user2_2
Combined features saved for 1_user9_2
Combined features saved for 0_user10
Combined features saved for 0_user10_2
Combined features saved for 1_user21
Combined features saved for 1_user2
Combined features saved for 0_user1
Combined features saved for 0_user3_2
Combined features saved for 1_user5
Combined features saved for 1_user5_2
Combined features saved for 1_user13
Combined features saved for 1_user19
Combined features saved for 1_user21_2
Feature combination completed!


/var/folders/fb/p7zywq8d7351kf4l4fwh2dmw0000gn/T/ipykernel_41381/1774875645.py:73: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  clip_data = torch.load(clip_path)


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
import os

# Paths
combined_features_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/hurdling/stages/stage3/combined_features"

# Hyperparameters
sequence_length = 20  # Adjust as needed (still remains as sequence length)
batch_size = 16       # Adjust as needed

# Ensure input size is always 515
input_size = 515

class AthleticsDataset(Dataset):
    def __init__(self, combined_features_dir, sequence_length, input_size):
        """
        Handles loading and processing of combined features for athletics data.
        Ensures input size is always 515.
        """
        self.data = []
        self.labels = []
        self.sequence_length = sequence_length
        self.input_size = input_size

        for file in os.listdir(combined_features_dir):
            if file.endswith("_combined.pt"):
                # Load combined features
                video_features = torch.tensor(torch.load(os.path.join(combined_features_dir, file)), dtype=torch.float32)

                # Truncate or pad sequences to the desired length
                if video_features.shape[0] >= self.sequence_length:
                    video_features = video_features[:self.sequence_length]
                else:
                    padding = torch.zeros((self.sequence_length - video_features.shape[0], video_features.shape[1]))
                    video_features = torch.cat((video_features, padding), dim=0)

                # Ensure features match the input size (515)
                if video_features.shape[1] != self.input_size:
                    padding = torch.zeros((video_features.shape[0], self.input_size - video_features.shape[1]))
                    video_features = torch.cat((video_features, padding), dim=1)
                self.data.append(video_features)

                # Extract label from filename
                label = float(file.split("_")[0])  # Extract label from file name
                self.labels.append(label)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Initialize dataset and DataLoader
def get_data_loaders(combined_features_dir, sequence_length, batch_size, input_size, train_split=0.8):
    dataset = AthleticsDataset(combined_features_dir, sequence_length, input_size)
    train_size = int(train_split * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, input_size  # Return loaders and input size

# Create DataLoaders
train_loader, val_loader, input_size = get_data_loaders(combined_features_dir, sequence_length, batch_size, input_size)

print(f"DataLoaders created. Input size: {input_size}")

DataLoaders created. Input size: 515


/var/folders/fb/p7zywq8d7351kf4l4fwh2dmw0000gn/T/ipykernel_41381/2703053424.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  video_features = torch.tensor(torch.load(os.

In [6]:
import torch.nn as nn

class TemporalModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        """
        LSTM-based model for sequence prediction.
        """
        super(TemporalModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)  # Use the final hidden state
        output = self.fc(hidden[-1])  # Fully connected output
        return output

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import itertools
import os

# Paths
combined_features_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/hurdling/stages/stage3/combined_features"

# Dataset and DataLoader
class AthleticsDataset(torch.utils.data.Dataset):
    def __init__(self, combined_features_dir, sequence_length):
        self.data = []
        self.labels = []
        self.sequence_length = sequence_length

        for file in os.listdir(combined_features_dir):
            if file.endswith("_combined.pt"):
                video_features = torch.tensor(torch.load(os.path.join(combined_features_dir, file)), dtype=torch.float32)

                if video_features.shape[0] >= self.sequence_length:
                    video_features = video_features[:self.sequence_length]
                else:
                    padding = torch.zeros((self.sequence_length - video_features.shape[0], video_features.shape[1]))
                    video_features = torch.cat((video_features, padding), dim=0)

                self.data.append(video_features)
                label = float(file.split("_")[0])  # Extract label from file name
                self.labels.append(label)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

def get_data_loaders(combined_features_dir, sequence_length, batch_size, train_split=0.8):
    dataset = AthleticsDataset(combined_features_dir, sequence_length)
    train_size = int(train_split * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, dataset[0][0].shape[1]

# Define LSTM model
class TemporalModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout):
        super(TemporalModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        output = self.fc(hidden[-1])
        return output

# Training function with early stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=500, patience=100):
    best_val_loss = float("inf")
    patience_counter = 0
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for features, labels in train_loader:
            features, labels = features.to(device), torch.tensor(labels, dtype=torch.float32).to(device)

            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for features, labels in val_loader:
                features, labels = features.to(device), torch.tensor(labels, dtype=torch.float32).to(device)
                outputs = model(features)
                loss = criterion(outputs.squeeze(), labels)
                val_loss += loss.item()

        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # # Early stopping logic with 100 epochs patience
        # if val_loss < best_val_loss:
        #     best_val_loss = val_loss
        #     patience_counter = 0
        # else:
        #     patience_counter += 1

        # if patience_counter >= patience:
        #     print(f"Early stopping triggered after {patience} epochs without improvement!")
        #     break

    return best_val_loss


# Hyperparameter grid search
# def hyperparameter_search():
    # Hyperparameter grid
    hidden_sizes = [32, 64, 128]
    num_layers = [1, 2, 3]
    learning_rates = [0.001, 0.005, 0.0001, 0.0005]
    dropouts = [0.0, 0.2, 0.3]

    # Initialize variables to track the best model
    best_val_loss = float("inf")
    best_params = None
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Prepare data
    train_loader, val_loader, input_size = get_data_loaders(combined_features_dir, sequence_length=20, batch_size=8)

    # Iterate over all combinations of hyperparameters
    for hidden_size, num_layer, learning_rate, dropout in itertools.product(hidden_sizes, num_layers, learning_rates, dropouts):
        print(f"Testing configuration: Hidden Size={hidden_size}, Num Layers={num_layer}, LR={learning_rate}, Dropout={dropout}")
        
        # Initialize model, criterion, and optimizer
        model = TemporalModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layer, output_size=1, dropout=dropout).to(device)
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        # Train the model
        val_loss = train_model(model, train_loader, val_loader, criterion, optimizer, device)

        # Update best model if this configuration is better
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_params = {
                "hidden_size": hidden_size,
                "num_layers": num_layer,
                "learning_rate": learning_rate,
                "dropout": dropout,
            }

    print(f"Best Configuration: {best_params}, Validation Loss: {best_val_loss:.4f}")
    return best_params

# Run hyperparameter search
# best_params = hyperparameter_search()

In [29]:
# Train the final model with the best configuration
final_hidden_size = 128
final_num_layers = 1
final_learning_rate = 0.001
final_dropout = 0.4

# Prepare DataLoaders (use full dataset for training)
train_loader, val_loader, input_size = get_data_loaders(combined_features_dir, sequence_length=20, batch_size=32)
print(f"Input size for the model: {input_size}")

# Initialize final model
device = "cuda" if torch.cuda.is_available() else "cpu"
final_model = TemporalModel(input_size=input_size, hidden_size=final_hidden_size, num_layers=final_num_layers, output_size=1, dropout=final_dropout).to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(final_model.parameters(), lr=final_learning_rate)

# Train final model
final_val_loss = train_model(final_model, train_loader, val_loader, criterion, optimizer, device, num_epochs=500, patience=100)
print(f"Final model trained. Validation Loss: {final_val_loss:.4f}")

/var/folders/fb/p7zywq8d7351kf4l4fwh2dmw0000gn/T/ipykernel_41381/3576269255.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  video_features = torch.tensor(torch.load(os.

Input size for the model: 513
Epoch [1/500], Train Loss: 0.6232, Val Loss: 0.5177
Epoch [2/500], Train Loss: 0.4572, Val Loss: 0.2618
Epoch [3/500], Train Loss: 0.4100, Val Loss: 0.1542
Epoch [4/500], Train Loss: 0.3690, Val Loss: 0.1188
Epoch [5/500], Train Loss: 0.3083, Val Loss: 0.1214
Epoch [6/500], Train Loss: 0.2458, Val Loss: 0.1447
Epoch [7/500], Train Loss: 0.1961, Val Loss: 0.1686
Epoch [8/500], Train Loss: 0.1606, Val Loss: 0.1714
Epoch [9/500], Train Loss: 0.1314, Val Loss: 0.1448
Epoch [10/500], Train Loss: 0.1031, Val Loss: 0.1039
Epoch [11/500], Train Loss: 0.0790, Val Loss: 0.0740
Epoch [12/500], Train Loss: 0.0618, Val Loss: 0.0668
Epoch [13/500], Train Loss: 0.0448, Val Loss: 0.0837
Epoch [14/500], Train Loss: 0.0285, Val Loss: 0.1305
Epoch [15/500], Train Loss: 0.0203, Val Loss: 0.1692
Epoch [16/500], Train Loss: 0.0173, Val Loss: 0.1527
Epoch [17/500], Train Loss: 0.0115, Val Loss: 0.1162
Epoch [18/500], Train Loss: 0.0103, Val Loss: 0.0987
Epoch [19/500], Train Los

In [30]:
import matplotlib.pyplot as plt

def evaluate_and_collect_predictions(model, data_loader, device):
    """
    Evaluate the model on the provided DataLoader and collect true vs. predicted values.
    """
    model.eval()
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for features, labels in data_loader:
            features = features.to(device)
            labels = labels.to(device)

            # Get model predictions
            outputs = model(features).squeeze()
            predicted_labels.extend(outputs.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    return true_labels, predicted_labels

# Use validation set for evaluation
true_labels, predicted_labels = evaluate_and_collect_predictions(final_model, val_loader, device)


In [31]:
import cv2
import os
import torch
from PIL import Image
import numpy as np
from transformers import CLIPProcessor, CLIPModel
import mediapipe as mp

# Paths
video_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/hurdling/stages/stage3/videos"
sequence_length = 20  # Sequence length for LSTM

# Initialize CLIP and MediaPipe
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, model_complexity=2, enable_segmentation=False)

# Ensure input size is always 515
input_size = 513

# Process and Predict for each video
def process_and_predict(video_path, model, sequence_length):
    print(f"Processing video: {os.path.basename(video_path)}")
    
    # Step 1: Extract frames from video
    frames = []
    cap = cv2.VideoCapture(video_path)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % 5 == 0:  # Extract frame every 5th frame
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(rgb_frame)
        count += 1
    cap.release()

    # Step 2: Extract pose keypoints using MediaPipe
    keypoints = []
    for frame in frames:
        result = pose.process(frame)
        if result.pose_landmarks:
            keypoints.append([
                {"x": lm.x, "y": lm.y, "z": lm.z, "visibility": lm.visibility}
                for lm in result.pose_landmarks.landmark
            ])
    
    # Step 3: Calculate velocity and acceleration (e.g., for legs, torso, or overall speed)
    velocities, accelerations = [], []
    for i in range(1, len(keypoints)):
        torso_prev = keypoints[i - 1][11]  # Left shoulder
        torso_curr = keypoints[i][11]  # Left shoulder
        velocity = np.sqrt((torso_curr['x'] - torso_prev['x'])**2 + (torso_curr['y'] - torso_prev['y'])**2)
        velocities.append(velocity)
        if i > 1:
            accelerations.append(velocities[-1] - velocities[-2])

    # Step 4: Assess hurdle clearance using angles (e.g., jump height, knee/leg angles)
    hurdle_clearance_angles = []
    for i in range(1, len(keypoints)):
        knee = keypoints[i][25]  # Left knee
        hip = keypoints[i][23]  # Left hip
        ankle = keypoints[i][27]  # Left ankle
        thigh_vector = np.array([knee['x'] - hip['x'], knee['y'] - hip['y']])
        shin_vector = np.array([ankle['x'] - knee['x'], ankle['y'] - knee['y']])
        
        dot_product = np.dot(thigh_vector, shin_vector)
        magnitude_thigh = np.linalg.norm(thigh_vector)
        magnitude_shin = np.linalg.norm(shin_vector)
        angle = np.arccos(dot_product / (magnitude_thigh * magnitude_shin))  # Knee angle during hurdle clearance
        hurdle_clearance_angles.append(np.degrees(angle))

    # Step 5: Extract CLIP embeddings for frames (semantic features)
    clip_embeddings = []
    for frame in frames:
        image = Image.fromarray(frame)
        inputs = clip_processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            embedding = clip_model.get_image_features(**inputs).cpu().numpy().flatten()
            clip_embeddings.append(embedding)

    # Step 6: Combine all features (CLIP, velocity, acceleration, hurdle clearance)
    combined_features = []
    for i, clip_embedding in enumerate(clip_embeddings):
        feature_vector = np.concatenate([clip_embedding, 
                                        [velocities[i] if i < len(velocities) else 0],
                                        [accelerations[i] if i < len(accelerations) else 0],
                                        [hurdle_clearance_angles[i] if i < len(hurdle_clearance_angles) else 0]])
        
        if len(feature_vector) > input_size:
            feature_vector = feature_vector[:input_size]  # Truncate if too long
        elif len(feature_vector) < input_size:
            padding = np.zeros(input_size - len(feature_vector))  # Pad with zeros if too short
            feature_vector = np.concatenate([feature_vector, padding])
        combined_features.append(feature_vector)
    
    # Convert to tensor
    combined_features = torch.tensor(combined_features, dtype=torch.float32)

    # Step 7: Truncate or pad sequences to the fixed sequence length
    if combined_features.shape[0] >= sequence_length:
        combined_features = combined_features[:sequence_length]
    else:
        padding = torch.zeros((sequence_length - combined_features.shape[0], combined_features.shape[1]))
        combined_features = torch.cat((combined_features, padding), dim=0)

    # Step 8: Predict using the model
    model.eval()
    with torch.no_grad():
        combined_features = combined_features.unsqueeze(0).to(device)  # Add batch dimension
        prediction = model(combined_features).squeeze().cpu().item()

    print(f"Prediction: {prediction:.4f}")
    return prediction


# Process all videos in the directory and predict
results = {}
for video_file in os.listdir(video_dir):
    if video_file.endswith(".mp4"):
        video_path = os.path.join(video_dir, video_file)
        prediction = process_and_predict(video_path, final_model, sequence_length)
        results[video_file] = prediction

# Print results
print("\nPrediction Results:")
for video, prediction in results.items():
    print(f"{video}: {prediction:.4f}")

I0000 00:00:1737034266.169644 4930217 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4 Pro
W0000 00:00:1737034266.227748 4948125 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737034266.254791 4948125 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Processing video: 1_user2.mp4
Prediction: 0.6425
Processing video: 1_user9_2.mp4
Prediction: 0.6079
Processing video: 1_user5.mp4
Prediction: 0.6423
Processing video: 1_user13.mp4
Prediction: 0.6473
Processing video: 1_user21_2.mp4
Prediction: 0.5183
Processing video: 0_user3.mp4
Prediction: 0.5392
Processing video: 0_user1.mp4
Prediction: 0.3578
Processing video: 0_user3_2.mp4
Prediction: 0.5850
Processing video: 0_user10.mp4
Prediction: 0.4030
Processing video: 1_user2_2.mp4
Prediction: 0.6348
Processing video: 0_user10_2.mp4
Prediction: 0.4314
Processing video: 1_user21.mp4
Prediction: 0.6590
Processing video: 1_user9.mp4
Prediction: 0.6433
Processing video: 1_user5_2.mp4
Prediction: 0.6314
Processing video: 1_user19.mp4
Prediction: 0.6063
Processing video: 1_user13_2.mp4
Prediction: 0.6731
Processing video: 0116.mp4
Prediction: 0.4030

Prediction Results:
1_user2.mp4: 0.6425
1_user9_2.mp4: 0.6079
1_user5.mp4: 0.6423
1_user13.mp4: 0.6473
1_user21_2.mp4: 0.5183
0_user3.mp4: 0.5392
0_

In [32]:
torch.save(final_model.state_dict(), "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/hurdling/stages/stage3/stage3_hurdles.pth")

___

## feedback generation

In [12]:
def process_video_with_metadata(video_path, model, sequence_length):
    """
    Process a video and return structured metadata with predictions and pose metrics.
    """
    print(f"Processing video: {os.path.basename(video_path)}")
    
    # Step 1: Extract frames
    frames = extract_frames(video_path)
    
    # Step 2: Extract keypoints
    keypoints = extract_keypoints(frames)

    # Step 3: Calculate pose-based features
    velocities, accelerations = calculate_velocity_and_acceleration(keypoints)
    stride_lengths = calculate_stride_length(keypoints)

    # Pose metrics
    pose_metrics = {
        "average_velocity": np.mean(velocities) if velocities else 0,
        "max_stride_length": np.max(stride_lengths) if stride_lengths else 0,
        "average_acceleration": np.mean(accelerations) if accelerations else 0
    }

    # Step 4: Extract CLIP features
    clip_embeddings = extract_clip_features(frames)

    # CLIP insights
    clip_summary = "High contextual alignment with accelerating motion."  # Placeholder for now

    # Step 5: Combine features
    combined_features = combine_features(clip_embeddings, velocities, accelerations, stride_lengths)

    # Step 6: Truncate or pad sequences
    if combined_features.shape[0] >= sequence_length:
        combined_features = combined_features[:sequence_length]
    else:
        padding = torch.zeros((sequence_length - combined_features.shape[0], combined_features.shape[1]))
        combined_features = torch.cat((combined_features, padding), dim=0)

    # Step 7: Predict using the model
    model.eval()
    with torch.no_grad():
        combined_features = combined_features.unsqueeze(0).to(device)  # Add batch dimension
        prediction = model(combined_features).squeeze().cpu().item()

    print(f"Prediction: {prediction:.4f}")

    # Return structured metadata
    return {
        "video_name": os.path.basename(video_path),
        "prediction": prediction,
        "pose_metrics": pose_metrics,
        "clip_features": {"embedding_summary": clip_summary}
    }

In [13]:
# from transformers import pipeline

# generator = pipeline("text2text-generation", model="google/flan-t5-base")
# result = generator("Explain the performance of an athlete based on metrics.", max_length=50)
# print(result[0]['generated_text'])


# def validate_metadata(metadata):
#     required_keys = {
#         'video_name': str,
#         'prediction': float,
#         'pose_metrics': dict,
#         'clip_features': dict,
#     }
#     pose_metrics_keys = ['average_velocity', 'max_stride_length', 'average_acceleration']
#     clip_features_keys = ['embedding_summary']

#     for key, expected_type in required_keys.items():
#         if key not in metadata or not isinstance(metadata[key], expected_type):
#             raise ValueError(f"Invalid or missing key: {key}, expected type: {expected_type}")
    
#     for key in pose_metrics_keys:
#         if key not in metadata['pose_metrics']:
#             raise ValueError(f"Missing pose metric: {key}")
    
#     for key in clip_features_keys:
#         if key not in metadata['clip_features']:
#             raise ValueError(f"Missing clip feature: {key}")

# def generate_justification(metadata, max_length=150, num_return_sequences=1):
#     validate_metadata(metadata)
    
#     # Dynamic prompt construction
#     metrics_prompt = []
#     for metric, value in metadata['pose_metrics'].items():
#         metrics_prompt.append(f"- {metric.replace('_', ' ').title()}: {value:.2f}")
#     metrics_text = "\n".join(metrics_prompt)

#     prompt = f"""
#     Analyze the performance for {metadata['video_name']}.
#     The predicted score is {metadata['prediction']:.2f}.
#     Key metrics:
#     {metrics_text}
#     - CLIP embedding summary: {metadata['clip_features']['embedding_summary']}
    
#     Based on these metrics, explain why the score is appropriate and provide constructive feedback for improvement.
#     """
#     print("Generated prompt:", prompt)  # Debugging log
#     result = generator(prompt, max_length=max_length, num_return_sequences=num_return_sequences)
#     return result[0]['generated_text']


In [14]:
# # Directory containing videos
# video_dir = "/Users/cezar/Desktop/Team Project/AI/distance_jump/stage1/videos"

# # Dictionary to store results
# results = {}

# for video_file in os.listdir(video_dir):
#     if video_file.endswith(".mp4"):
#         video_path = os.path.join(video_dir, video_file)
        
#         # Step 1: Process video and generate metadata
#         metadata = process_video_with_metadata(video_path, final_model, sequence_length=20)
        
#         # Step 2: Generate justification using Hugging Face model
#         justification = generate_justification(metadata)
#         metadata["justification"] = justification
        
#         # Store results
#         results[video_file] = metadata

# # Print results
# for video, data in results.items():
#     print(f"Video: {video}")
#     print(f"Prediction: {data['prediction']:.4f}")
#     print(f"Justification: {data['justification']}")
#     print("-" * 50)

# # Optional: Save results to JSON
# # import json
# # with open("video_predictions_with_justifications.json", "w") as f:
# #     json.dump(results, f, indent=4)